Data Loading

In [210]:
import pandas as pd
events_data=pd.read_csv("events_data.csv")
products_data=pd.read_csv("products_data.csv")

Merge the two dataframes

In [211]:
merged_data = pd.merge(products_data, events_data, left_on='ext_id', right_on='product_id', how='left')

Timestamp to time conversion

In [212]:
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], unit='s', errors='coerce')
merged_data['time'] = merged_data['timestamp'].dt.time
merged_data = merged_data.drop(columns=['timestamp'])

Replacing na with unisex

In [213]:
merged_data['ontology'] = merged_data['ontology'].str.replace('na>', 'Unisex>')

Splitting ontology into categories and product

In [214]:
merged_data[['Gender', 'Category1', 'Category2']] = merged_data['ontology'].str.split('>', n=2, expand=True)
merged_data['Before_First'] = merged_data['Category2'].str.split('>').str[0]
merged_data['After_Last'] = merged_data['Category2'].str.rsplit('>', n=1).str[-1]
merged_data.drop(columns=['ontology','Category1','Category2'], inplace=True)
merged_data.rename(columns={'Before_First': 'category', 'After_Last': 'product'}, inplace=True)
merged_data.fillna(value='None', inplace=True)

ext_id                  0
price                   0
discounted_price        0
date                  842
mad_uuid              842
event                 842
page_type             842
product_id            842
product_price       80215
module_id           76789
time                  842
Gender                  0
category              446
product               446
dtype: int64


Filling missing price values with mean price of a category

In [215]:
merged_data['price'].replace(-1.0, pd.NA, inplace=True)
category_mean_prices = merged_data.groupby('category')['price'].mean()

In [216]:
def replace_nan_with_mean(row):
    category_mean_price = category_mean_prices.get(row['category'])
    if pd.isna(row['price']) and category_mean_price is not None:
        return category_mean_price
    else:
        return row['price']

merged_data['price'] = merged_data.apply(replace_nan_with_mean, axis=1)

In [217]:
merged_data['discounted_price'] = merged_data.apply(
    lambda row: (row['price'] + row['discounted_price']) if row['discounted_price'] < 0 else row['discounted_price'],
    axis=1
)

In [218]:
merged_data

,ext_id,price,discounted_price,date,mad_uuid,event,page_type,product_id,product_price,module_id,time,Gender,category,product
0,4089,108.439647,108.003300,None,None,None,None,None,None,None,None,Unisex,Skincare,Moisturisers
1,426,70.000000,66.258557,None,None,None,None,None,None,None,None,Unisex,Skincare,Face Toners
2,394,70.000000,64.957784,None,None,None,None,None,None,None,None,Unisex,Skincare,Face Cleanser and Face Wash
3,490,43.900000,26.820353,None,None,None,None,None,None,None,None,Male,Deodorant,Deodorant
4,500,155.000000,103.990141,None,None,None,None,None,None,None,None,Unisex,Fragrance,Eau De Toilette
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85741,83616,52.000000,22.539083,2022-01-03,3ba9efcc-3322-4650-beb2-3392a2d83cd8,pageView - pageView,home,83616,None,None,04:09:00,Female,Body Cleanser,Body Cleansers
85742,83616,52.000000,22.539083,2022-01-04,9a61a5c1-3013-4192-a6ae-46b02bc93c9f,pageView - pageView,pdp,83616,None,None,15:54:02,Female,Body Cleanser,Body Cleansers
85743,83616,52.000000,22.539083,2022-01-01,e1f19162-0c87-4829-9bf2-a72861c4f560,pageView - pageView,cat,83616,None,None,12:38:09,Female,Body Cleanser,Body Cleansers
85744,83616,52.000000,22.539083,2022-01-02,119b563d-bcbe-402c-8418-4b730ddda574,pageView - pageView,pdp,83616,None,None,20:45:32,Female,Body Cleanser,Body Cleansers


Convert to excel

In [219]:
merged_data.to_excel('cleaned.xlsx')

For recommendation module - Merge data based on the mad_uuid users who purchased by recommendation modules

In [220]:
filtered_data = merged_data.loc[merged_data['module_id']!='None',['mad_uuid','module_id']]

In [221]:
order_confirmation_data = merged_data[merged_data['event'] == 'orderConfirmation - pageView']

In [222]:
result = pd.merge(filtered_data, order_confirmation_data, on='mad_uuid', how='inner')
result=result[['mad_uuid','module_id_x','product_price','category','product']]

In [223]:
result

,mad_uuid,module_id_x,product_price,category,product
0,772eb602-3734-405c-b7f7-89c2e81c0c2a,3509.0,42.0,Bath & Body,Body Butter
1,772eb602-3734-405c-b7f7-89c2e81c0c2a,3509.0,42.0,Bath & Body,Body Butter
2,772eb602-3734-405c-b7f7-89c2e81c0c2a,3509.0,42.0,Bath & Body,Body Butter
3,772eb602-3734-405c-b7f7-89c2e81c0c2a,3509.0,42.0,Bath & Body,Body Butter
4,772eb602-3734-405c-b7f7-89c2e81c0c2a,3509.0,42.0,Bath & Body,Body Butter
...,...,...,...,...,...
2456,54e4b58a-ae1c-4312-acd4-851b2be9707e,3213.0,155.0,Fragrance,Eau De Toilette
2457,d97011ac-f9d7-4123-b474-f3f0eafedd96,3509.0,155.0,Fragrance,Eau De Toilette
2458,c85a1993-af82-4633-a89b-a0aff955704c,3216.0,55.0,Body,Hair Treatment & Styling
2459,c85a1993-af82-4633-a89b-a0aff955704c,3216.0,55.0,Body,Hair Treatment & Styling


Convert to excel

In [224]:
result.to_excel('recommendation.xlsx')